QC Array
========

***Analysis step***

**Author:** *Jay Kim*

## Test sample contamination (bafRegress)
G. Jun, M. Flickinger, K. N. Hetrick, Kurt, J. M. Romm, K. F. Doheny, G. Abecasis, M. Boehnke,and H. M. Kang, _Detecting and Estimating Contamination of Human DNA Samples in Sequencing and Array-Based Genotype Data_, American journal of human genetics doi:10.1016/j.ajhg.2012.09.004 (volume 91 issue 5 pp.839 - 848)

In [ ]:
library(httr)
library(jsonlite)
library(dplyr)

Aggregate batch MAFs across entire study:

In [48]:
json <- content(GET("http://{endpoint}/maf?batchid=all"), "text")
# json <-
# '[
#     {"JIRA" : "JIRA_1", "sample_count" : 4, "mafs" : [0.4324,0.3,0.234,0.05]},
#     {"JIRA" : "JIRA_2", "sample_count" : 4, "mafs" : [0.4012,0.3,0.4,0.5]}
# ]'
batches <- fromJSON(json)
mafs <- bind_cols(batches$mafs)
names(mafs) <- batches$JIRA
maf <- rowSums(mafs*batches$sample_count)/sum(batches$sample_count)

Warning message:
“package ‘httr’ was built under R version 4.0.5”
New names:
• `` -> `...1`
• `` -> `...2`


[1] 0.4168 0.3000 0.3170 0.2750

Get sample info for batch and test each sample for contamination:

In [52]:
json <- content(GET("http://{endpoint}/sampleinfo?batchid={JIRA}"), "text")
sample_info <- fromJSON(json)
# json <-
# '[
#     {"JIRA" : "JIRA_1", "sample_id" : "CPT1", "barcode" : "xxxxxx", "position" : "RXXCXX"},
#     {"JIRA" : "JIRA_2", "sample_id" : "CPT2", "barcode" : "xxxxxx", "position" : "RXXCXX"}
# ]'
getgtc <- function(sample) {
    json <- content(GET("http://{endpoint}/gtc/{sample$barcode}_{sample$position}"), "text")
    return(fromJSON(json))
}
source("bafRegress.R")
baf_results <- testsamples(sample_info, maf, getgtc)

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Could not resolve host: {endpoint}


## Import Packages

In [ ]:
source("analysis-utils.R")

library(optparse)
library(kableExtra)
library(NMF)
library(RColorBrewer)
library(plyr)
library(dplyr)
library(SNPRelate)
library(argyle)

set the name of comparison prefix by pulling the parent directory name

In [ ]:
comparison = tail(strsplit(getwd(),"/")[[1]],n=1)

## Import .idat

In [ ]:
raw_data <- import.idat(final_report_prefix = strsplit(grep("FinalReport.txt",
                                                            grep("contam",list.files(),value=T,invert=T),
                                                            value=T),
                                                       split="_FinalReport.txt")
                        )

## QC .idat

In [ ]:
# ...

## Import .gds file

In [ ]:
# ...